# Introdução ao LangSmith

O LangSmith é uma plataforma para testar, depurar e avaliar aplicativos baseados em LLM.
Neste notebook, vamos explorar os conceitos fundamentais:
1. **Tracing**: Como visualizar o fluxo de execução das suas chains.
2. **Datasets**: Como criar conjuntos de dados para testes.
3. **Avaliação**: Como rodar testes automatizados nos seus datasets.


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

%pip install -qU langsmith langchain langchain-openai

In [ ]:
import os
import getpass

# LangSmith Config
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

if not os.environ.get("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangSmith API Key: ")

if not os.environ.get("LANGCHAIN_PROJECT"):
    os.environ["LANGCHAIN_PROJECT"] = "curso-langsmith-intro" # Nome do projeto no LangSmith

# OpenAI API Key
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key: ")

## 1. Tracing Básico

O tracing já está ativo porque definimos `LANGCHAIN_TRACING_V2="true"`. Qualquer chamada feita com componentes do LangChain será registrada automaticamente.

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

# Simples chamada, isso vai gerar um trace
response = llm.invoke("Explique o que é LangSmith em uma frase.")
print(response.content)

Vá para [smith.langchain.com](https://smith.langchain.com) e verifique o projeto `curso-langsmith-intro`. Você deve ver o trace da execução acima.

## 2. Datasets

Datasets são coleções de exemplos (entradas e saídas esperadas) usados para testar e avaliar seu aplicativo.

In [ ]:
from langsmith import Client

client = Client()

dataset_name = "Perguntas de Exemplo"

# Criar Dataset se não existir
if not client.has_dataset(dataset_name=dataset_name):
    dataset = client.create_dataset(
        dataset_name=dataset_name,
        description="Um dataset de exemplo com perguntas e respostas."
    )
    
    # Adicionar exemplos
    client.create_examples(
        inputs=[
            {"pergunta": "Qual é a capital da França?"},
            {"pergunta": "Quanto é 2 + 2?"},
            {"pergunta": "Quem escreveu Dom Casmurro?"}
        ],
        outputs=[
            {"resposta": "Paris"},
            {"resposta": "4"},
            {"resposta": "Machado de Assis"}
        ],
        dataset_id=dataset.id
    )
    print("Dataset criado!")
else:
    print("Dataset já existe.")

## 3. Avaliação

Agora vamos usar o dataset criado para avaliar nosso modelo.

In [ ]:
from langchain.smith import RunEvalConfig, run_on_dataset

# 1. Definir o que vamos testar (nosso "sistema")
# O sistema deve aceitar a entrada do dataset e produzir uma saída.

def meu_app(inputs):
    return llm.invoke(inputs["pergunta"]).content

# 2. Configurar Avaliadores
# Avaliadores "QA" verificam a precisão comparando com a resposta de referência (output do dataset)
eval_config = RunEvalConfig(
    evaluators=["qa"], # usa um LLM para julgar se a resposta bate com o gabarito
)

# 3. Rodar Avaliação
results = run_on_dataset(
    client=client,
    dataset_name=dataset_name,
    llm_or_chain_factory=meu_app,
    evaluation=eval_config,
)

print(results)

Agora você pode visualizar os resultados da avaliação no painel do LangSmith.